#### Gradient Descent

In [7]:
import random

def step(v, direction, step_size):
    """move step_size in the direction from v"""
    return [v_i + step_size * direction_i for v_i, direction_i in zip(v, direction)]

def sum_of_squares_gradient(v):
    return [2 * v_i for v_i in v]

def distance(v1, v2):
    """Euclidean distance between two vectors."""
    return sum((v1_i - v2_i) ** 2 for v1_i, v2_i in zip(v1, v2)) ** 0.5

# pick a random starting point
v = [random.randint(-10, 10) for _ in range(3)]
print(v)
tolerance = 0.0000001

while True:
    gradient = sum_of_squares_gradient(v)  # compute the gradient at v
    next_v = step(v, gradient, -0.01)      # take a negative gradient step
    if distance(next_v, v) < tolerance:    # stop if we're converging
        break
    v = next_v

print("Minimum point:", v)


[-1, -4, 4]
Minimum point: [-8.65402814934158e-07, -3.461611259736632e-06, 3.461611259736632e-06]


In [8]:
def safe(f):
    """return a new function that's the same as f,
    except that it outputs infinity whenever f produces an error"""
    def safe_f(*args, **kwargs):
        try:
            return f(*args, **kwargs)
        except:
            return float('inf') # this means "infinity" in Python
    return safe_f

### Minimize Batch

In [10]:
def minimize_batch(target_fn, gradient_fn, theta_0, tolerance=0.000001):
    """use gradient descent to find theta that minimizes target function"""
    step_sizes = [100, 10, 1, 0.1, 0.01, 0.001, 0.0001, 0.00001]
    theta = theta_0 # set theta to initial value
    target_fn = safe(target_fn) # safe version of target_fn
    value = target_fn(theta) # value we're minimizing
    while True:
        gradient = gradient_fn(theta)
        next_thetas = [step(theta, gradient, -step_size)
        for step_size in step_sizes]
        # choose the one that minimizes the error function
        next_theta = min(next_thetas, key=target_fn)
        next_value = target_fn(next_theta)
        # stop if we're "converging"
        if abs(value - next_value) < tolerance:
            return theta
        else:
            theta, value = next_theta, next_value

### Minimize Stochastic

In [11]:
def in_random_order(data):
    """generator that returns the elements of data in random order"""
    indexes = [i for i, _ in enumerate(data)] # create a list of indexes
    random.shuffle(indexes) # shuffle them
    for i in indexes: # return the data in that order
        yield data[i]

In [12]:
def multiply_scalar(c, vector):
    return [c * v_i for v_i in vector]
multiply_scalar(9, [7, 4])

[63, 36]

In [16]:
def vector_subtract(v, w):
    return [v_i - v_j for v_i, v_j in zip(v, w)]
vector_subtract([2,  4, 5], [6, 8, 9])

[-4, -4, -4]

In [15]:
def minimize_stochastic(target_fn, gradient_fn, x, y, theta_0, alpha_0=0.01):
    data = zip(x, y)
    theta = theta_0 # initial guess
    alpha = alpha_0 # initial step size
    min_theta, min_value = None, float("inf") # the minimum so far
    iterations_with_no_improvement = 0
    # if we ever go 100 iterations with no improvement, stop
    while iterations_with_no_improvement < 100:
        value = sum( target_fn(x_i, y_i, theta) for x_i, y_i in data )
        if value < min_value:
        # if we've found a new minimum, remember it
        # and go back to the original step size
            min_theta, min_value = theta, value
            iterations_with_no_improvement = 0
            alpha = alpha_0
    else:
        # otherwise we're not improving, so try shrinking the step size
        iterations_with_no_improvement += 1
        alpha *= 0.9
    # and take a gradient step for each of the data points
    for x_i, y_i in in_random_order(data):
        gradient_i = gradient_fn(x_i, y_i, theta)
        theta = vector_subtract(theta, multiply_scalar(alpha, gradient_i))
    return min_theta